# Nemerical Method for Pricing Barrier Options

## Monte Carlo Simulation

#### As a reminder
$$ S_{t+\Delta t} = S_t \exp \left[ \left( \mu - \frac{\sigma ^2}{2} \right) \Delta t + \sigma \epsilon \sqrt{\Delta t} \right]$$

In [50]:
import numpy as np
import pandas as pd
np.random.seed(0)

In [51]:
steps = 252 # Number of steps excluding the initial day
T = 1
r = 0.03
q = 0
sig = 0.13
simulations = 100000

S = 1
B2 = 1.03 * S # Upperbound for strike-out
B1 = 0.8 * S # Lowerbound for strike-in
strike_out_interest = 0.2
div_payment = 0.2 # Dividend payment (Annual)

In [52]:
def MC_path(S, T, r, q, sig, steps, simulations):
    delta_t = T/steps
    price = np.zeros((steps + 1, simulations)) # Including the initial price
    price[0] = S
    for i in range(1, steps + 1):
        z = np.random.standard_normal(simulations) # N times of sample epsilon~N(0, 1), where N=simulations
        price[i] = price[i - 1] * np.exp(((r-q) - 0.5 * sig **2) * delta_t + sig * np.sqrt(delta_t) * z)
    return price.T


def MC_pricing(path, B1, B2, strike_out_interest, div_payment, r):
    value = None
    down_flag = False
    up_flag = False
    days = np.arange(0, path.shape[0], 1)

    up_B2 = (path >= B2)
    strike_out_observe = (days % 21 == 0)
    strike_out_days = days[up_B2 * strike_out_observe] # On strike-out observation day & strike-out
    strike_in_days = days[path < B1]

    up_flag = len(strike_out_days) > 0 
    down_flag = len(strike_in_days) > 0
    # Once strike-out on observation day, the option ceases
    if up_flag: 
        end_day = strike_out_days[0] # The first observed strike-out day
        value = strike_out_interest * (end_day / 252) * np.exp(-r*end_day / 252)
    # No strike-out on oberservation days and ever strike-in
    elif len(strike_out_days) == 0 and len(strike_in_days) > 0: 
        end_day = 252
        if path[-1] < path[0]:
            value = (path[-1] - path[0]) * np.exp(-r*end_day/252)
        else:
            value = 0
    elif len(strike_out_days) == 0 and len(strike_in_days) == 0: # Never strike-in or strike-out
        end_day = 252
        value = div_payment * (end_day / 252) * np.exp(-r*end_day / 252)
    return value

In [53]:
paths = MC_path(S, T, r, q, sig, steps, simulations)
value_list = []
for i in range(len(paths)):
    path = paths[i]
    value = MC_pricing(path, B1, B2, strike_out_interest, div_payment, r)
    value_list.append(value)
print(np.mean(value_list))

0.07369910838725921


## Finite Difference Method

#### An overview of the Crank-Nicholson method is presented as the following, yet the other two methods would be also included in the algorithm.

#### Crank-Nicolson method is more stable than the explicit method and quickly converges to the solution.
#### Derivative terms thus become
$$\frac{\partial f}{\partial S_t} = \frac{f_{i, j+1} - f_{i, j-1}}{2 \Delta S_t} \Rightarrow \frac{\partial f}{\partial S_t} \approx \frac{1}{2} \left( \frac{f_{i+1, j+1} - f_{i+1, j-1}}{2\Delta S_t} + \frac{f_{i, j+1} - f_{i, j-1}}{2\Delta S_t}\right)$$
$$\frac{\partial^2 f}{\partial S_t^2} = \frac{f_{i, j+1} - 2f_{i, j} + f_{i, j-1}}{\Delta S_t ^2} \Rightarrow \frac{\partial^2 f}{\partial S_t^2} \approx \frac{1}{2} \left( \frac{f_{i+1, j+1} + f_{i+1, j-1} - 2f_{i+1, j}}{\Delta S_t^2} + \frac{f_{i, j+1} + f_{i, j-1} - 2f_{i, j}}{\Delta S_t^2}\right)$$

#### The equation is thus simplified to 
$$-\alpha_j f_{i, j-1} + (1-\beta_j)f_{i, j} - \gamma_j f_{i, j+1} = \alpha_j f_{i+1, j-1} + (1+\beta_j)f_{i+1, j} + \gamma_j f_{i+1, j+1}$$
#### where
$$\alpha_j = \frac{\Delta t}{4}(\sigma^2j^2 - rj)$$
$$\beta_j = -\frac{\Delta t}{2}(\sigma^2j^2 + r)$$
$$\gamma_j = \frac{\Delta t}{4}(\sigma^2j^2 + rj)$$

#### The matrix form thus become
$$M_1f_i = M_2f_{i+1} + b$$
#### where $f_i$ and $b$ are vectors of length $M-1$
$$f_i = \left[f_{i, 1}, f_{i, 2}, ..., f_{i, M-1}\right]^T$$
$$b = \left[\alpha_1(f_{i, 0} + f_{i+1, 0}, 0, ..., 0, \gamma_{M-1}(f_{i, M} + f_{i+1, M})\right]^T$$
#### and $M_1$, $M_2$ are matrix of dimensions $(M-1) \times (M-2)$.
$$ M_1 = \begin{bmatrix}
1-\beta_1 & -\gamma_1 & 0 & ... & 0 \\
-\alpha_2 & 1-\beta_2 & -\gamma_2 & ... & 0 \\
0 & -\alpha 3 & ... & ... & ... \\
... & ... & ... & 1-\beta_{M-2} & -\gamma_{M-2} \\
0 & 0 & ... & -\alpha_{M-1} & 1-\beta_{M-1} 
\end{bmatrix}  $$
$$ M_2 = \begin{bmatrix}
1+\beta_1 & \gamma_1 & 0 & ... & 0 \\
\alpha_2 & 1+\beta_2 & \gamma_2 & ... & 0 \\
0 & \alpha 3 & ... & ... & ... \\
... & ... & ... & 1+\beta_{M-2} & \gamma_{M-2} \\
0 & 0 & ... & \alpha_{M-1} & 1+\beta_{M-1} 
\end{bmatrix}  $$
#### Thus, the iteration becomes
$$f_i = M_1^{-1}M_2f_{i+1} + M_1^{-1}b$$

#### We may break down a barrier option (Snowball) into a combinations of four types of distinct barrier options: 
#### an Up-and-In Option, a Double Knock-Out Option, an Up-and-Out Put Option, and a Double Knock-Out Option, 
#### denoting them as OTU, DNT, DKOP, and UOP. 
#### Then, the price of a complex barrier option can be expressed as an arithmetic operation: OTU + DNT + DKOP - UOP.

### OTU

Such options take effect when the price of the underlying asset touches or exceeds a preset high threshold. In 
Snowball bonds, if the price of basic assets rises to a certain level, additional payments or more favorable return conditions are activated.

#### First define relevant paramters

In [2]:
M = 500 # Grid Numbers
N = 252 # Time Steps

S0 = 1 # Base Price
SU = 1.03 # Strike-Out Bound
SD = 0.85 # Strike-In Bound

time = 1 # Total Time to Maturity (One Year)
delta_t = time / N
delta_s = S0 / M
price_array = np.linspace(0, M*delta_s, M+1)
time_array = np.inspace(0, N*delta_t, N+1)
time_to_maturity = time_array[-1]  - time_array

sigma = 0.13 # Volatility
r = 0.03 # Risk-Free Rate
b = 0.03 
Div = 0.20 # Dividend Payment Rate

strike_out_days = np.arange(21, time*252+1, 21) # Days for Strike-Out Observation

In [12]:
def get_diff(j, sig=0.4):
    discount = 
    a = -0.25 * rjt + 0.25 * sigma2j2t
    b = -0.5 * (self.r-self.q) * self.delta_T - 0.5 * sigma2j2t
    c = 0.25 * rjt + 0.25 * sigma2j2t
return a, b, c
    

def gen_matrix(SD_idx, M, a, b, c):
    a_m_1 = [a(i) for i in range(Sd_idx + 2,Sd_idx + M)]
    b_m_1 = [b(i) for i in range(Sd_idx + 1,Sd_idx + M)]
    c_m_1 = [c(i) for i in range(Sd_idx + 1,Sd_idx+ M-1)]
    diag_matrix = np.diag(a_m_1,-1) + np.diag(b_m_1,0) + np.diag(c_m_1,1)
    return diag_matrix


def calculate_OTU(S0, time, sigma, r, b, M, N, SU, Div):
    matrix = np.zeros(((int(SU/ds) * 3) + 1, N+1)) 
    KO = np.where(price_array

SyntaxError: incomplete input (3258688888.py, line 53)

In [7]:
a, b, c = get_diff(3)
gen_matrix(0, M, a, b, c)

NameError: name 'delta_t' is not defined

In [9]:
def num(method):
    if method == '3':
        return 3
    else:
        return 5